<a href="https://colab.research.google.com/github/thiagorainmaker77/paper/blob/master/analise_liar_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install zeugma


In [4]:
import pandas as pd
import sklearn as skl
import seaborn as sb
import re
import gensim
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from zeugma.embeddings import EmbeddingTransformer

import warnings
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler



from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy
import numpy as np

from sklearn.metrics import accuracy_score


from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.metrics import f1_score


import gensim
import numpy as np


warnings.filterwarnings('ignore')

In [2]:
import torch

if torch.cuda.is_available :
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)  

cuda


#Util

In [5]:

# %matplotlib inline


class Thiago_Mestrado_Oracle(object):

    def __init__(self):
        self.rng = 42

    def get_coefficients(self, y_true, y_pred_a, y_pred_b):
        a, b, c, d = 0, 0, 0, 0
        for i in range(y_true.shape[0]):
            if (y_pred_a[i] == y_true[i]) & (y_pred_b[i] == y_true[i]):
                a = a + 1
            elif (y_pred_a[i] != y_true[i]) & (y_pred_b[i] == y_true[i]):
                b = b + 1
            elif (y_pred_a[i] == y_true[i]) & (y_pred_b[i] != y_true[i]):
                c = c + 1
            else:
                d = d + 1

        return a, b, c, d

    def double_fault_measure(self, y_true, y_pred_a, y_pred_b):
        a, b, c, d = self.get_coefficients(y_true, y_pred_a, y_pred_b)
        return float(d) / (a + b + c + d)



    def monta_oracle(self, label, pre={}, sz_w=5, sz_h=5, titulo=''):

        oracle = []
        grafico = []

        for i in range(len(label)):
            oracle.append(0)

        for i in range(len(pre)):
            grafico.append(0)

        for x in pre:
            i = 0
            for y in pre[x]:
                if (label[i] == y):
                    oracle[i] = oracle[i] + 1
                i = i + 1

        erros = 0;
        acertos = 0;
        for x in oracle:
            if (x == 0):
                erros = erros + 1
            else:
                acertos = acertos + 1

        barras = {}
        for x in oracle:
            if (x == 0):
                continue
            grafico[x - 1] = grafico[x - 1] + 1

        label = []
        k = 1
        for x in grafico:
            label.append(k)
            k = k + 1
        index = np.arange(len(label))
        plt.figure(figsize=(sz_w, sz_h))
        plt.bar(index, grafico)
        plt.xlabel('Acerto por classificador', fontsize=10)
        plt.xticks(index, label, fontsize=10)
        plt.title(titulo+'total de acertos em conjunto:' + str(acertos))
        plt.show()


    def monta_tsne(self, classificadores, statement, label, title='', predicoes={}, escala=500, size=5, x_ini=0, x_fim=0, y_ini=0, y_fim=0):



        predicoes_test = {}
        for x in classificadores:
            predicoes[x] = classificadores[x].predict(statement)

        m = len(predicoes)
        n = len(predicoes)

        delta = [[0 for x in range(n)] for x in range(m)]
        labels = []

        i = 0
        for x in predicoes:
            k = 0
            labels.append(x)
            for y in predicoes:
                delta[i][k] = self.double_fault_measure(label.array, predicoes[x], predicoes[y])
                k = k + 1
            i = i + 1

        delta = np.array(delta)

        tsne_model = TSNE(perplexity=50, init='pca', n_iter=2500, random_state=23)
        new_values = tsne_model.fit_transform(delta)

        x = []
        y = []
        for value in new_values:
            x.append(value[0])
            y.append(value[1])

        plt.figure(figsize=(size, size))
        plt.title(title)

        if escala > 0:
          plt.ylim(escala * -1, escala)
          plt.xlim(escala * -1, escala)
        else:
          plt.ylim(y_ini, y_fim)
          plt.xlim(x_ini, x_fim)

        simbolos = {}
        simbolos[0] = 'X'
        simbolos[1] = 'd'
        simbolos[2] = '*'
        simbolos[3] = "^"
        simbolos[4] = 'o'
        
        dot = 0
        for i in range(len(x)):
            plt.scatter(x[i], y[i], marker=simbolos[dot], c='#808080', s=100)
            print(labels[i], simbolos[dot])
            #plt.annotate(labels[i], xy=(x[i], y[i]), xytext=(5, 2), textcoords='offset points', ha='center', va='bottom')
            dot = dot+1
        plt.show()


    def monta_tsne_full(self, classificadores, statement, label, title='', predicoes={}, escala=500, size=5, x_ini=0, x_fim=0, y_ini=0, y_fim=0):



        predicoes_test = {}
        for x in classificadores:
            predicoes[x] = classificadores[x].predict(statement)

        m = len(predicoes)
        n = len(predicoes)

        delta = [[0 for x in range(n)] for x in range(m)]
        labels = []

        i = 0
        for x in predicoes:
            k = 0
            labels.append(x)
            for y in predicoes:
                delta[i][k] = self.double_fault_measure(label.array, predicoes[x], predicoes[y])
                k = k + 1
            i = i + 1

        delta = np.array(delta)

        tsne_model = TSNE(perplexity=50, init='pca', n_iter=2500, random_state=23)
        new_values = tsne_model.fit_transform(delta)

        x = []
        y = []
        for value in new_values:
            x.append(value[0])
            y.append(value[1])

        plt.figure(figsize=(size, size))
        plt.title(title)

        if escala > 0:
          plt.ylim(escala * -1, escala)
          plt.xlim(escala * -1, escala)
        else:
          plt.ylim(y_ini, y_fim)
          plt.xlim(x_ini, x_fim)

        simbolos = {}
        simbolos[0] = 'X'
        simbolos[1] = 'd'
        simbolos[2] = '*'
        simbolos[3] = "^"
        simbolos[4] = 'o'
        
        dot = 0
        for i in range(len(x)):
            plt.scatter(x[i], y[i] , c='#808080', s=10)
            plt.annotate(labels[i], xy=(x[i], y[i]), xytext=(5, 2), textcoords='offset points', ha='center', va='bottom')
            dot = dot+1
        plt.show()



    def monta_tsne_pred(self,statement, label, title='', predicoes={}, escala=500, size=5, x_ini=0, x_fim=0, y_ini=0, y_fim=0):


        m = len(predicoes)
        n = len(predicoes)

        delta = [[0 for x in range(n)] for x in range(m)]
        labels = []

        i = 0
        for x in predicoes:
            k = 0
            labels.append(x)
            for y in predicoes:
                delta[i][k] = self.double_fault_measure(label.array, predicoes[x], predicoes[y])
                k = k + 1
            i = i + 1

        delta = np.array(delta)

        tsne_model = TSNE(perplexity=50, init='pca', n_iter=2500, random_state=23)
        new_values = tsne_model.fit_transform(delta)

        x = []
        y = []
        for value in new_values:
            x.append(value[0])
            y.append(value[1])

        plt.figure(figsize=(size, size))
        plt.title(title)

        if escala > 0:
          plt.ylim(escala * -1, escala)
          plt.xlim(escala * -1, escala)
        else:
          plt.ylim(y_ini, y_fim)
          plt.xlim(x_ini, x_fim)

        simbolos = {}
        simbolos[0] = 'X'
        simbolos[1] = 'd'
        simbolos[2] = '*'
        simbolos[3] = "^"
        simbolos[4] = 'o'
        
        dot = 0
        for i in range(len(x)):
            plt.scatter(x[i], y[i], marker=simbolos[dot], c='#808080', s=100)
            print(labels[i], simbolos[dot])
            #plt.annotate(labels[i], xy=(x[i], y[i]), xytext=(5, 2), textcoords='offset points', ha='center', va='bottom')
            dot = dot+1
        plt.show()

In [35]:

# %matplotlib inline

# teste
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.items())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


class Mestrado_Thiago_util(object):

    def __init__(self):
        self.rng = 42

    def get_dataset(self):
        uri_train = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'
        uri_valid = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/valid.tsv'
        uri_test = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'
        df_train = pd.read_table(uri_train,
                                 names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                        'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                        'venue'])
        df_valid = pd.read_table(uri_valid,
                                 names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                        'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                        'venue'])
        df_test = pd.read_csv(uri_test, sep='\t',
                              names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                     'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                     'venue'])
        df = pd.concat([df_train, df_valid])

        return df_train, df_valid, df_test, df

    def get_coefficients(self, y_true, y_pred_a, y_pred_b):
        a, b, c, d = 0, 0, 0, 0
        for i in range(y_true.shape[0]):
            if (y_pred_a[i] == y_true[i]) & (y_pred_b[i] == y_true[i]):
                a = a + 1
            elif (y_pred_a[i] != y_true[i]) & (y_pred_b[i] == y_true[i]):
                b = b + 1
            elif (y_pred_a[i] == y_true[i]) & (y_pred_b[i] != y_true[i]):
                c = c + 1
            else:
                d = d + 1

        return a, b, c, d

    def double_fault_measure(self, y_true, y_pred_a, y_pred_b):
        a, b, c, d = get_coefficients(y_true, y_pred_a, y_pred_b)
        return float(d) / (a + b + c + d)

    def monta_oracle(self, label, predicoes={}, sz=5):

        oracle = []
        grafico = []

        for i in range(len(label)):
            oracle.append(0)

        for i in range(len(predicoes)):
            grafico.append(0)

        for x in predicoes:
            i = 0
            for y in predicoes[x]:
                if (label[i] == y):
                    oracle[i] = oracle[i] + 1
                i = i + 1

        erros = 0;
        acertos = 0;
        for x in oracle:
            if (x == 0):
                erros = erros + 1
            else:
                acertos = acertos + 1

        barras = {}
        for x in oracle:
            if (x == 0):
                continue
            grafico[x - 1] = grafico[x - 1] + 1

        label = []
        k = 1
        for x in grafico:
            label.append(k)
            k = k + 1
        index = np.arange(len(label))
        # plt.figure(figsize=(sz+3, sz))
        plt.bar(index, grafico)
        plt.xlabel('Acerto por classificador', fontsize=10)
        plt.xticks(index, label, fontsize=10)
        plt.title('Total de acertos em conjunto:' + str(acertos))
        plt.show()


#https://github.com/thiagorainmaker77/libs.git





    def get_perceptron(self, statement, label, glove, w2v, fasttext):
        lr1 = Perceptron(random_state=42)

        lr2 = Perceptron(random_state=42)

        lr3 = Perceptron(random_state=42)

        lr4 = Perceptron(random_state=42)

        lr5 = Perceptron(random_state=42)

        countV = CountVectorizer()
        countV.fit_transform(statement)

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        lr_cv = Pipeline([
            ('NBCV', countV),
            ('clf', lr1)])

        lr_tfid = Pipeline([
            ('NBCV', tfidf_ngram),
            ('clf', lr2)])

        lr_w2v = Pipeline([
            ('NBCV', w2v),
            ('clf', lr3)])

        lr_glove = Pipeline([
            ('NBCV', glove),
            ('clf', lr4)])

        lr_fast = Pipeline([
            ('NBCV', fasttext),
            ('clf', lr5)])

        lr_cv.fit(statement, label)
        lr_tfid.fit(statement, label)
        lr_w2v.fit(statement, label)
        lr_glove.fit(statement, label)
        lr_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = lr_cv
        classificadores["TFIDF"] = lr_tfid
        classificadores["Word2Vec"] = lr_w2v
        classificadores["Glove"] = lr_glove
        classificadores["FastText"] = lr_fast
        return classificadores





    def get_pac(self, statement, label, glove, w2v, fasttext):
        lr1 = PassiveAggressiveClassifier(random_state=42)

        lr2 = PassiveAggressiveClassifier(random_state=42)

        lr3 = PassiveAggressiveClassifier(random_state=42)

        lr4 = PassiveAggressiveClassifier(random_state=42)

        lr5 = PassiveAggressiveClassifier(random_state=42)

        countV = CountVectorizer()
        countV.fit_transform(statement)

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        lr_cv = Pipeline([
            ('NBCV', countV),
            ('clf', lr1)])

        lr_tfid = Pipeline([
            ('NBCV', tfidf_ngram),
            ('clf', lr2)])

        lr_w2v = Pipeline([
            ('NBCV', w2v),
            ('clf', lr3)])

        lr_glove = Pipeline([
            ('NBCV', glove),
            ('clf', lr4)])

        lr_fast = Pipeline([
            ('NBCV', fasttext),
            ('clf', lr5)])

        lr_cv.fit(statement, label)
        lr_tfid.fit(statement, label)
        lr_w2v.fit(statement, label)
        lr_glove.fit(statement, label)
        lr_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = lr_cv
        classificadores["TFIDF"] = lr_tfid
        classificadores["Word2Vec"] = lr_w2v
        classificadores["Glove"] = lr_glove
        classificadores["FastText"] = lr_fast
        return classificadores





    def get_knn(self, statement, label, glove, w2v, fasttext):
        lr1 = KNeighborsClassifier()

        lr2 = KNeighborsClassifier()

        lr3 = KNeighborsClassifier()

        lr4 = KNeighborsClassifier()

        lr5 = KNeighborsClassifier()

        countV = CountVectorizer()
        countV.fit_transform(statement)

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        lr_cv = Pipeline([
            ('NBCV', countV),
            ('clf', lr1)])

        lr_tfid = Pipeline([
            ('NBCV', tfidf_ngram),
            ('clf', lr2)])

        lr_w2v = Pipeline([
            ('NBCV', w2v),
            ('clf', lr3)])

        lr_glove = Pipeline([
            ('NBCV', glove),
            ('clf', lr4)])

        lr_fast = Pipeline([
            ('NBCV', fasttext),
            ('clf', lr5)])

        lr_cv.fit(statement, label)
        lr_tfid.fit(statement, label)
        lr_w2v.fit(statement, label)
        lr_glove.fit(statement, label)
        lr_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = lr_cv
        classificadores["TFIDF"] = lr_tfid
        classificadores["Word2Vec"] = lr_w2v
        classificadores["Glove"] = lr_glove
        classificadores["FastText"] = lr_fast
        return classificadores

    def get_lr(self, statement, label, glove, w2v, fasttext):

        lr1 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                                 intercept_scaling=1, max_iter=100, 
                                 n_jobs=None, penalty='l2',
                                 random_state=self.rng, tol=0.0001, verbose=0, warm_start=False)

        lr2 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                                 intercept_scaling=1, max_iter=100, 
                                 n_jobs=None, penalty='l2',
                                 random_state=self.rng, tol=0.0001, verbose=0, warm_start=False)

        lr3 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                                 intercept_scaling=1, max_iter=100,
                                 n_jobs=None, penalty='l2',
                                 random_state=self.rng, tol=0.0001, verbose=0, warm_start=False)

        lr4 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                                 intercept_scaling=1, max_iter=100, 
                                 n_jobs=None, penalty='l2',
                                 random_state=self.rng, tol=0.0001, verbose=0, warm_start=False)

        lr5 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                                 intercept_scaling=1, max_iter=100, 
                                 n_jobs=None, penalty='l2',
                                 random_state=self.rng, tol=0.0001, verbose=0, warm_start=False)

        countV = CountVectorizer(analyzer='word', binary=False, decode_error='strict', encoding='utf-8',
                                 input='content',
                                 lowercase=False, max_df=0.5, max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None, stop_words='english',
                                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        lr_cv = Pipeline([
            ('NBCV', countV),
            ('clf', lr1)])

        lr_tfid = Pipeline([
            ('NBCV', tfidf_ngram),
            ('clf', lr2)])

        lr_w2v = Pipeline([
            ('NBCV', w2v),
            ('clf', lr3)])

        lr_glove = Pipeline([
            ('NBCV', glove),
            ('clf', lr4)])

        lr_fast = Pipeline([
            ('NBCV', fasttext),
            ('clf', lr5)])

        lr_cv.fit(statement, label)
        lr_tfid.fit(statement, label)
        lr_w2v.fit(statement, label)
        lr_glove.fit(statement, label)
        lr_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = lr_cv
        classificadores["TFIDF"] = lr_tfid
        classificadores["Word2Vec"] = lr_w2v
        classificadores["Glove"] = lr_glove
        classificadores["FastText"] = lr_fast
        return classificadores

    
  

    
    
    
    
    def get_sgd(self, statement, label, glove, w2v, fasttext):

        countV = CountVectorizer(analyzer='word', binary=False, decode_error='strict', encoding='utf-8',
                                 input='content',
                                 lowercase=False, max_df=0.5, max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None, stop_words='english',
                                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        sgd_cv = Pipeline([
            ('sgd_tfidf', countV),
            ('sgd_clf', SGDClassifier(loss='log', penalty='l2', alpha=1e-3, random_state=self.rng))
        ])

        sgd_tf = Pipeline([
            ('sgd_tfidf', tfidf_ngram),
            ('sgd_clf', SGDClassifier(loss='log', penalty='l2', random_state=self.rng, alpha=1e-3))
        ])

        sgd_w2v = Pipeline([
            ('sgd_tfidf', w2v),
            ('sgd_clf', SGDClassifier(loss='log', penalty='l2', alpha=1e-3, random_state=self.rng))
        ])

        sgd_glove = Pipeline([
            ('sgd_tfidf', glove),
            ('sgd_clf', SGDClassifier(loss='log', penalty='l2', alpha=1e-3, random_state=self.rng))
        ])

        sgd_fast = Pipeline([
            ('sgd_tfidf', fasttext),
            ('sgd_clf', SGDClassifier(loss='log', penalty='l2', alpha=1e-3, random_state=self.rng))
        ])

        sgd_cv.fit(statement, label)
        sgd_tf.fit(statement, label)
        sgd_w2v.fit(statement, label)
        sgd_glove.fit(statement, label)
        sgd_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = sgd_cv
        classificadores["TFIDF"] = sgd_tf
        classificadores["Word2Vec"] = sgd_w2v
        classificadores["Glove"] = sgd_glove
        classificadores["FastText"] = sgd_fast

        return classificadores

    def get_svm(self, statement, label, glove, w2v, fasttext):

        countV = CountVectorizer(stop_words='english')
        countV.fit_transform(statement)

        svm.SVC

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        svm_cv = Pipeline([
            ('svmCV', countV),
            ('svm_clf', svm.SVC(probability=True, random_state=self.rng))
        ])

        svm_tf = Pipeline([
            ('svmCV', tfidf_ngram),
            ('svm_clf', svm.SVC(probability=True, random_state=self.rng))
        ])

        svm_w2v = Pipeline([
            ('sgd_tfidf', w2v),
            ('sgd_clf', svm.SVC(probability=True, random_state=self.rng))
        ])

        svm_glove = Pipeline([
            ('sgd_tfidf', glove),
            ('sgd_clf', svm.SVC(probability=True, random_state=self.rng))
        ])

        svm_fast = Pipeline([
            ('sgd_tfidf', fasttext),
            ('sgd_clf', svm.SVC(probability=True, random_state=self.rng))
        ])

        svm_cv.fit(statement, label)
        svm_tf.fit(statement, label)
        svm_w2v.fit(statement, label)
        svm_glove.fit(statement, label)
        svm_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = svm_cv
        classificadores["TFIDF"] = svm_tf
        classificadores["Word2Vec"] = svm_w2v
        classificadores["Glove"] = svm_glove
        classificadores["FastText"] = svm_fast

        return classificadores



    def get_svm_linear_svc(self, statement, label, glove, w2v, fasttext):

        countV = CountVectorizer(stop_words='english')
        countV.fit_transform(statement)

        svm.SVC

        tfidf_ngram = TfidfVectorizer(stop_words='english')
        tfidf_ngram.fit_transform(statement)

        svm_cv = Pipeline([
            ('svmCV', countV),
            ('svm_clf', svm.LinearSVC(random_state=self.rng))
        ])

        svm_tf = Pipeline([
            ('svmCV', tfidf_ngram),
            ('svm_clf', svm.LinearSVC(random_state=self.rng))
        ])

        svm_w2v = Pipeline([
            ('sgd_tfidf', w2v),
            ('sgd_clf', svm.LinearSVC(random_state=self.rng))
        ])

        svm_glove = Pipeline([
            ('sgd_tfidf', glove),
            ('sgd_clf', svm.LinearSVC(random_state=self.rng))
        ])

        svm_fast = Pipeline([
            ('sgd_tfidf', fasttext),
            ('sgd_clf', svm.LinearSVC(random_state=self.rng))
        ])

        svm_cv.fit(statement, label)
        svm_tf.fit(statement, label)
        svm_w2v.fit(statement, label)
        svm_glove.fit(statement, label)
        svm_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = svm_cv
        classificadores["TFIDF"] = svm_tf
        classificadores["Word2Vec"] = svm_w2v
        classificadores["Glove"] = svm_glove
        classificadores["FastText"] = svm_fast

        return classificadores        

    def get_extraTrees(self, statement, label, glove, w2v, fasttext):
        cv_ex = CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                                encoding='utf-8', input='content',
                                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                                tokenizer=None, vocabulary=None)

        ex_cv1 = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                                     min_impurity_decrease=0.0, min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
                                     oob_score=False, random_state=self.rng, verbose=0, warm_start=False)

        ex_cv2 = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
                                      oob_score=False, random_state=self.rng, verbose=0, warm_start=False)

        ex_cv3 = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
                                      oob_score=False, random_state=self.rng, verbose=0, warm_start=False)

        ex_cv4 = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
                                      oob_score=False, random_state=self.rng, verbose=0, warm_start=False)

        ex_cv5 = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
                                      oob_score=False, random_state=self.rng, verbose=0, warm_start=False)

        extp_cv = Pipeline([
            ('NBCV', cv_ex),
            ('nb_clf', ex_cv1)])

        tf_ex = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                                encoding='utf-8', input='content',
                                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                                ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
                                stop_words='english', strip_accents=None, sublinear_tf=False,
                                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                                vocabulary=None)

        extp_tf = Pipeline([
            ('NBCV', tf_ex),
            ('nb_clf', ex_cv2)])

        extp_w2 = Pipeline([
            ('NBCV', w2v),
            ('nb_clf', ex_cv3)])

        extp_glove = Pipeline([
            ('NBCV', glove),
            ('nb_clf', ex_cv4)])

        extp_fast = Pipeline([
            ('NBCV', fasttext),
            ('nb_clf', ex_cv5)])

        extp_cv.fit(statement, label)
        extp_tf.fit(statement, label)
        extp_w2.fit(statement, label)
        extp_glove.fit(statement, label)
        extp_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = extp_cv
        classificadores["TFIDF"] = extp_tf
        classificadores["Word2Vec"] = extp_w2
        classificadores["Glove"] = extp_glove
        classificadores["FastText"] = extp_fast

        return classificadores

    
    def get_rfc(self, statement, label, glove, w2v, fasttext):

        cv_rf = CountVectorizer()

        rf_c = RandomForestClassifier(random_state=42)

        rf_c1 = RandomForestClassifier(random_state=42)

        rf_c2 = RandomForestClassifier(random_state=42)

        rf_c3 = RandomForestClassifier(random_state=42)

        rf_tf = RandomForestClassifier(random_state=42)

        rfp_cv = Pipeline([
            ('NBCV', cv_rf),
            ('nb_clf', rf_c)])

        tf_cv = TfidfVectorizer()

      
        

        rfp_tf = Pipeline([
            ('NBCV', tf_cv),
            ('nb_clf', rf_tf)])

        rf_w2v = Pipeline([
            ('NBCV', w2v),
            ('clf', rf_c1)])

        rf_glove = Pipeline([
            ('NBCV', glove),
            ('clf', rf_c2)])

        rf_fast = Pipeline([
            ('NBCV', fasttext),
            ('clf', rf_c3)])

        rfp_cv.fit(statement, label)
        rfp_tf.fit(statement, label)
        rf_w2v.fit(statement, label)
        rf_glove.fit(statement, label)
        rf_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = rfp_cv
        classificadores["TFIDF"] = rfp_tf
        classificadores["Word2Vec"] = rf_w2v
        classificadores["Glove"] = rf_glove
        classificadores["FastText"] = rf_fast

        return classificadores

    def get_mlp(self, statement, label, glove, w2v, fasttext):
        cv_ex = CountVectorizer()

        tf_cv = TfidfVectorizer()

        mpl_cv = Pipeline([
            ('svmCV', cv_ex),
            ('svm_clf', MLPClassifier(random_state=self.rng))
        ])

        mpl_tf = Pipeline([
            ('svmCV', tf_cv),
            ('svm_clf', MLPClassifier(random_state=self.rng))
        ])

        mpl_w2v = Pipeline([
            ('sgd_tfidf', w2v),
            ('sgd_clf', MLPClassifier(random_state=self.rng))
        ])

        mpl_glove = Pipeline([
            ('sgd_tfidf', glove),
            ('sgd_clf', MLPClassifier(random_state=self.rng))
        ])

        mpl_fast = Pipeline([
            ('sgd_tfidf', fasttext),
            ('sgd_clf', MLPClassifier(random_state=self.rng))
        ])

        mpl_cv.fit(statement, label)
        mpl_tf.fit(statement, label)
        mpl_w2v.fit(statement, label)
        mpl_glove.fit(statement, label)
        mpl_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = mpl_cv
        classificadores["TFIDF"] = mpl_tf
        classificadores["Word2Vec"] = mpl_w2v
        classificadores["Glove"] = mpl_glove
        classificadores["FastText"] = mpl_fast

        return classificadores


    def get_multinomial_nb(self, statement, statement_teste, label, glove, w2v, fasttext):


        mnb_cv = Pipeline([
            ('1', CountVectorizer()),
            ('2', MultinomialNB())])

        mnb_tf = Pipeline([
            ('1', TfidfVectorizer()),
            ('2', MultinomialNB())])

        mnb_w2v = Pipeline([
            ('1', w2v),
            ('2', MinMaxScaler(feature_range=(0, 1))),
            ('3', MultinomialNB())])

        mnb_glove = Pipeline([
            ('1', glove),
            ('2', MinMaxScaler(feature_range=(0, 1))),
            ('3', MultinomialNB())])

        mnb_fast = Pipeline([
            ('1', fasttext),
            ('2', MinMaxScaler(feature_range=(0, 1))),
            ('3', MultinomialNB())])

        mnb_cv.fit(statement, label)
        mnb_tf.fit(statement, label)
        mnb_w2v.fit(statement, label)
        mnb_glove.fit(statement, label)
        mnb_fast.fit(statement, label)

        classificadores = {}

        classificadores["CV"] = mnb_cv
        classificadores["TFIDF"] = mnb_tf
        classificadores["Word2Vec"] = mnb_w2v
        classificadores["Glove"] = mnb_glove
        classificadores["FastText"] = mnb_fast

        return classificadores


In [13]:
df_train, df_valid, df_test, df = mestrado_util.get_dataset()

In [15]:
treinamento = pd.DataFrame(columns=['statement', 'label', 'label_binario'])

i = 0
for index, row in df_train.iterrows():
  label = row['label']
  binario = '0'
  if label == 'true' or label == 'mostly-true' or label ==  'half-true' :
    binario = '1'
    
  treinamento = treinamento.append({'statement': row['statement'], 'label': label, 'label_binario': binario }, ignore_index=True)
  
  
for index, row in df_valid.iterrows():
  label = row['label']
  binario = '0'
  if label == 'true' or label == 'mostly-true' or label ==  'half-true' :
    binario = '1'
  
  treinamento = treinamento.append({'statement': row['statement'], 'label': label, 'label_binario': binario }, ignore_index=True)
  
  
teste = pd.DataFrame(columns=['statement', 'label', 'label_binario'])
df_test['label_binario'] = ' - '
i = 0
for index, row in df_test.iterrows():
  label = row['label']
  binario = '0'
  if label == 'true' or label == 'mostly-true' or label ==  'half-true' :
    binario = '1'
  
  df_test.at[index,'label_binario']=binario
  teste = teste.append({'statement': row['statement'], 'label': label, 'label_binario': binario }, ignore_index=True)

#cod

In [36]:
mestrado_oracle = Thiago_Mestrado_Oracle()
mestrado_util = Mestrado_Thiago_util()

In [11]:
w2v = EmbeddingTransformer('word2vec')
glove = EmbeddingTransformer('glove')
fasttext = EmbeddingTransformer('fasttext')

[=================================================-] 99.7% 1657.1/1662.8MB downloaded
[==================================================] 100.0% 104.8/104.8MB downloaded
[=================================================-] 98.8% 947.1/958.4MB downloaded

## classificadores

In [16]:
svm_c = mestrado_util.get_svm_linear_svc(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [17]:
mnb = mestrado_util.get_multinomial_nb(treinamento['statement'], teste['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [19]:
sgd = mestrado_util.get_sgd(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [20]:
pca = mestrado_util.get_pac(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [21]:
perceptron = mestrado_util.get_perceptron(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [22]:
ext = mestrado_util.get_extraTrees(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [23]:
rf = mestrado_util.get_rfc(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [24]:
mlp = mestrado_util.get_mlp(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [37]:
lr = mestrado_util.get_lr(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [38]:
knn = mestrado_util.get_knn(treinamento['statement'], treinamento['label_binario'], glove, w2v, fasttext)

In [39]:
results = pd.DataFrame(columns=['Técnica', 'CV', 'TFIDF',  'W2V', 'GloVe', 'Fasttext'])
combinacao = pd.DataFrame(columns=['Técnica', 'Voto', 'Average'])